# Zadanie 1

In [59]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at /tmp/ipykernel_11581/427647947.py:3 

In [60]:
from pyspark.sql.functions import *

filepath = "online-retail-dataset.csv"
df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load(filepath)

## NULLs
### Fill

In [61]:
df.na.fill(0).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

### Explode

In [62]:
df.withColumn("Splitted", split(col("Country"), " ")) \
  .withColumn("Exploded", explode(col("Splitted"))) \
  .show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------------+--------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|         Splitted|Exploded|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------------+--------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|[United, Kingdom]|  United|
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|[United, Kingdom]| Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|[United, Kingdom]|  United|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|[United, Kingdom]| Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|Uni

### Drop

In [63]:
print(f"DataFrame count before: {df.count()}")
print(f"DataFrame count after : {df.na.drop().count()}")

DataFrame count before: 541909


DataFrame count after : 406829


### Regex Replace

In [64]:
df.select(col("Country"),regexp_replace(col("Country"), r"United Kingdom", "Scotland").alias("Country Replaced")).show()

+--------------+----------------+
|       Country|Country Replaced|
+--------------+----------------+
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
+--------------+----------------+
only showing top 20 rows



### Regex Extract

In [65]:
df.select(regexp_extract(col("Country"), r"(United)", 1).alias("Country Extracted")).show()

+-----------------+
|Country Extracted|
+-----------------+
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
+-----------------+
only showing top 20 rows



### If Null

In [66]:
#% sql

### Null If


In [67]:
#% sql

### Replace

In [68]:
df.na.replace(2.55, 0.).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|      0.0|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

### Array Contains

In [69]:
df.withColumn("Is UK", array_contains(split(col("Country"), " "), "United")).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Is UK|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom| true|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom| true|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom| true|
|   536365|    2173

# Zadanie 2

## Numeric UDF

In [73]:
from pyspark.sql.types import DoubleType, IntegerType
import numpy as np


def rounded(col):
    return np.round(col)


def add_thousand(col):
    return col + 1000

udf_round = udf(rounded, DoubleType())
udf_add_thousand = udf(add_thousand, IntegerType())

In [74]:
df.withColumn("QuantityLarge", udf_add_thousand(col("Quantity"))).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|QuantityLarge|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|         1006|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|         1006|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|         1008|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|         1006|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|         1006|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:2

In [75]:
df.withColumn("UnitPriceRound", udf_round(col("UnitPrice"))).show()

23/04/11 21:52:06 ERROR Executor: Exception in task 0.0 in stage 59.0 (TID 55)
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$Lambda$3542/1938500683.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	a

Py4JJavaError: An error occurred while calling o802.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 55) (192.168.0.102 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala

## String UDF

In [76]:
from pyspark.sql.types import StringType

def to_lower(string):
    return string.lower()

udf_to_lower = udf(to_lower, StringType())

In [77]:
df.withColumn("DescriptionLowered", udf_to_lower(col("Description"))).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|  DescriptionLowered|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|white hanging hea...|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| white metal lantern|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|cream cupid heart...|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|knitted union fla...|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|red woolly hottie...|
|   5363